In [6]:
import string
# load data
filename = 'shakespeare-DATA.txt'
#file = open(filename, encoding="iso-8859-1")
file = open(filename, encoding="utf-8")

text = file.read()
file.close()



In [4]:
#remove stop words, apply stemming on tokens and generate frequency of tokens for a given text.
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
def create_frequency_table(text_string) -> dict:

    stopWords = set(stopwords.words("english"))
    words = word_tokenize(text_string)
    ps = PorterStemmer()
    freqTable = dict()

    for word in words:

        word = ps.stem(word)

        if word in stopWords:

            continue

        if word in freqTable:

            freqTable[word] += 1

        else:

            freqTable[word] = 1

    return freqTable

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\smart\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
from itertools import islice
freqTable=create_frequency_table(text)
print("\n \n frequency Table \n \n",list(islice(freqTable.items(), 50)))


 
 frequency Table 
 
 [('[', 2), ('emma', 855), ('jane', 301), ('austen', 1), ('1816', 1), (']', 2), ('volum', 3), ('I', 3177), ('chapter', 57), ('woodhous', 312), (',', 12016), ('handsom', 42), ('clever', 31), ('rich', 14), ('comfort', 120), ('home', 132), ('happi', 198), ('disposit', 25), ('seem', 212), ('unit', 13), ('best', 83), ('bless', 30), ('exist', 9), (';', 2353), ('live', 79), ('nearli', 14), ('twenty-on', 1), ('year', 83), ('world', 82), ('veri', 1202), ('littl', 362), ('distress', 32), ('vex', 5), ('.', 6355), ('wa', 2396), ('youngest', 4), ('two', 178), ('daughter', 66), ('affection', 10), ('indulg', 24), ('father', 201), ('consequ', 33), ('sister', 44), ("'s", 866), ('marriag', 36), ('mistress', 12), ('hi', 1141), ('hous', 100), ('earli', 41), ('period', 19)]


In [7]:
#cleaning text
import string
from nltk.tokenize import sent_tokenize

def cleaned_text(text):
    text = text.replace('\n', '')
    text = text.replace('[', '')
    text = text.replace(']', '')
    text = re.sub('\d', '',text)
    text = re.sub('\s+',' ',text)
    
    return text

In [8]:
#split into sentences(tokenized sentences)
Sentences=sent_tokenize(text)

print("\n Sentences: \n ",Sentences[:20])


 Sentences: 
  ['[Emma by Jane Austen 1816]\n\nVOLUME I\n\nCHAPTER I\n\n\nEmma Woodhouse, handsome, clever, and rich, with a comfortable home\nand happy disposition, seemed to unite some of the best blessings\nof existence; and had lived nearly twenty-one years in the world\nwith very little to distress or vex her.', "She was the youngest of the two daughters of a most affectionate,\nindulgent father; and had, in consequence of her sister's marriage,\nbeen mistress of his house from a very early period.", 'Her mother\nhad died too long ago for her to have more than an indistinct\nremembrance of her caresses; and her place had been supplied\nby an excellent woman as governess, who had fallen little short\nof a mother in affection.', "Sixteen years had Miss Taylor been in Mr. Woodhouse's family,\nless as a governess than a friend, very fond of both daughters,\nbut particularly of Emma.", 'Between _them_ it was more the intimacy\nof sisters.', "Even before Miss Taylor had ceased to hold 

In [9]:
#Finding score of a sentence by adding up all the frequencies of words(after removing the stop words) contained in it.

def score_sentences(sentences, freqTable) -> dict:
    sentenceValue = dict()

    for sentence in sentences:
        word_count_in_sentence = (len(word_tokenize(sentence)))
        for wordValue in freqTable:
            if wordValue in sentence.lower():
                if sentence[:10] in sentenceValue:
                    sentenceValue[sentence[:10]] += freqTable[wordValue]
                else:
                    sentenceValue[sentence[:10]] = freqTable[wordValue]

        sentenceValue[sentence[:10]] = sentenceValue[sentence[:10]] // word_count_in_sentence

    return sentenceValue

In [10]:
sentenceValue = score_sentences(Sentences, freqTable)
print(" \n \n sentenceValue \n \n",sentenceValue)

 
 
 sentenceValue 
 
 {'[Emma by J': 445, 'She was th': 1959, 'Her mother': 518, 'Sixteen ye': 780, 'Between _t': 881, 'Even befor': 405, 'The real e': 653, 'The danger': 840, 'Sorrow cam': 479, 'It was Mis': 1089, 'It was on ': 654, 'The weddin': 471, 'Her father': 728, 'The event ': 1059, 'Mr. Weston': 501, 'The want o': 734, 'She recall': 426, 'A large de': 576, 'She had be': 921, 'How was sh': 426, 'She dearly': 1580, 'He could n': 1139, 'The evil o': 384, 'Her sister': 404, 'Highbury, ': 1269, 'The Woodho': 908, 'All looked': 1109, 'She had\nma': 695, 'It was a m': 567, 'His spirit': 1525, 'He was a n': 765, 'Matrimony,': 277, 'Emma smile': 1448, 'What a pit': 711, '"I cannot ': 567, '"A house o': 262, 'This is th': 1339, '"How often': 561, '_We_ must ': 682, '"My dear, ': 2020, 'Randalls i': 1079, 'I could no': 629, '"No, papa,': 1954, 'We must go': 1638, '"The carri': 297, 'But James ': 334, '"They are ': 749, 'You know w': 806, 'We talked ': 754, 'And as for': 946, 'I only dou

In [11]:
#Finding average score of all sentences, so that it can be used as a threshold to 
#integrate all the sentences for which sentenceValue is above threshold value to form a summary.

def find_average_score(sentenceValue) -> int:
    sumValues = 0

    for entry in sentenceValue:
        sumValues += sentenceValue[entry]


    # Average value of a sentence from original text

    average = int(sumValues / len(sentenceValue))



    return average

In [12]:
threshold = find_average_score(sentenceValue)
print(" \n \n threshold \n \n",threshold)

 
 
 threshold 
 
 968


In [13]:
#If the sentence score is more than the average score then select the sentence to form a summary

def generate_summary(sentences, sentenceValue, threshold):

    sentence_count = 0
    summary = ''


    for sentence in sentences:

        if sentence[:10] in sentenceValue and sentenceValue[sentence[:10]] > (threshold):

            summary += " " + sentence

            sentence_count += 1

    return summary

In [15]:

# 5 Important Algorithm: Generate the summary

summary = generate_summary(Sentences, sentenceValue, 1.5 * threshold)

print(summary)

 She was the youngest of the two daughters of a most affectionate,
indulgent father; and had, in consequence of her sister's marriage,
been mistress of his house from a very early period. She dearly loved her father, but he
was no companion for her. His spirits required support. "My dear, how am I to get so far? "No, papa, nobody thought of your walking. We must go in the carriage,
to be sure." That was
your doing, papa. It was a happy circumstance, and animated Mr. Woodhouse for some time. "Dirty, sir! said Emma playfully. "Dear Emma bears every thing so well," said her father. Oh dear, no! "Only one more, papa; only for Mr. Elton. Poor Mr. Elton! "Mr. Elton is a very pretty young man, to be sure, and a very
good young man, and I have a great regard for him. That will be a much better thing. From the expense of the child, however, he was soon relieved. Mr. Frank Churchill was one of the boasts of Highbury, and a lively
curiosity to see him prevailed, though the compliment was so littl

Parts of speech tagging can be important for syntactic and semantic analysis.
Morpho-syntactic tagging, also known as grammatical tagging or POS-tagging, is a process that consists of automatically associating words in a text with corresponding grammatical information: verbs, adjectives, nouns, gender, number, etc.

In [17]:
# Demonstrating POS tagging if in case we are trying to determine the abstractive way of summary for a given document.

import nltk

nltk.download('punkt')
# required for parts of speech tagging
nltk.download('averaged_perceptron_tagger')
# parts of speech tagging
#split into words(tokenize)
from nltk.tokenize import word_tokenize
tokens = word_tokenize(text)
tagged = nltk.pos_tag(tokens)
# print tagged tokens
print(tagged[:50])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\smart\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\smart\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


[('[', 'NNS'), ('Emma', 'NNP'), ('by', 'IN'), ('Jane', 'NNP'), ('Austen', 'NNP'), ('1816', 'CD'), (']', 'NNP'), ('VOLUME', 'NNP'), ('I', 'PRP'), ('CHAPTER', 'VBP'), ('I', 'PRP'), ('Emma', 'NNP'), ('Woodhouse', 'NNP'), (',', ','), ('handsome', 'NN'), (',', ','), ('clever', 'NN'), (',', ','), ('and', 'CC'), ('rich', 'JJ'), (',', ','), ('with', 'IN'), ('a', 'DT'), ('comfortable', 'JJ'), ('home', 'NN'), ('and', 'CC'), ('happy', 'JJ'), ('disposition', 'NN'), (',', ','), ('seemed', 'VBD'), ('to', 'TO'), ('unite', 'VB'), ('some', 'DT'), ('of', 'IN'), ('the', 'DT'), ('best', 'JJS'), ('blessings', 'NNS'), ('of', 'IN'), ('existence', 'NN'), (';', ':'), ('and', 'CC'), ('had', 'VBD'), ('lived', 'VBN'), ('nearly', 'RB'), ('twenty-one', 'CD'), ('years', 'NNS'), ('in', 'IN'), ('the', 'DT'), ('world', 'NN'), ('with', 'IN')]
